# IRF - Uppsala Python Workshop: Snakes in Space 🐍
author: Louis Richard
e-mail: louisr@irfu.se
date: 29/02/2024

## Time series, spectrogram, VDFs, & co.
Introduction to data structures for space plasma analysis:
    - The xarray package\
    - DataArrays for time series and spectrograms.\
    - Datasets for velocity distribution functions.